In [1]:
#import seaborn as sn
import wfdb
from wfdb import processing
import os
import glob
import pandas as pd
import numpy as np
#%matplotlib notebook
import matplotlib.pyplot as plt
from scipy import signal

In [2]:
### 128 ticks per second NOT 250
fs = 128
fs_target = 250


records_nsr = []
properties_nsr = []
annot_nsr = []
AnnSymb_nsr = []
AnnSamp_nsr = []
AnnRhythm_nsr = []
#Rpeak_Samp = []
#Rpeak_Symb = []
new_records_nsr = []
AnnSamp_250_nsr = []

for f in glob.glob('C:/Users/masud/Skrivebord/Github_Bachelor_Project/wfdb-python/normal-sinus-rhythm-data/*.dat'): ##### change the path (afib) to your own directory.
    sig, fields = wfdb.rdsamp(f[:-4], channels=[1])  #### In this function, you have the opportunity to pass "channels=[0]" or "channels=[1]" to select channel 1 or 2.
    
#    x, _ = processing.resample_sig(x=sig[:,0], fs=fields['fs'], fs_target=250)
#    new_sig = processing.resample_sig(x=sig, fs=128, fs_target=250)
#    new_sig = signal.resample(sig, 250)
    
    ann = wfdb.rdann(f[:-4], 'atr')
#    QRS = wfdb.rdann(f[:-4], 'qrs') ### xws extension instead
    Symb = pd.Series(ann.symbol)
    Samp = pd.Series(ann.sample)
#    x = processing.resample_sig(Samp, fs=fields['fs'], fs_target=250)
#    QRS_Symb = pd.Series(QRS.symbol)
#    QRS_Samp = pd.Series(QRS.sample)
    Rhythm = pd.Series(ann.aux_note)
    records_nsr.append(sig)
    properties_nsr.append(fields)
    annot_nsr.append(ann)
    AnnSymb_nsr.append(Symb)
    AnnSamp_nsr.append(Samp)
#    AnnSamp_250_nsr.append(x)
#    Rpeak_Symb.append(QRS_Symb)
#    Rpeak_Samp.append(QRS_Samp)
    AnnRhythm_nsr.append(Rhythm)
    
    

AnnSymb_nsr = pd.Series(AnnSymb_nsr).values
AnnSamp_nsr = pd.Series(AnnSamp_nsr).values

# Pre processing data...

In [3]:
labeled_Rpeaks_nsr = []
appended_data_nsr = []
for i in range(18): ## 18 records
    df = pd.DataFrame(AnnSamp_nsr[i]*2) ### Roughly resampling from 128 to 250 Hz
    appended_data_nsr.append(df)
        

labeled_Rpeaks_nsr = pd.concat(appended_data_nsr)
#labeled_Rpeaks_nsr

In [4]:
labeled_Rpeaks_nsr.rename(columns={0: 'Rpeaks'},
          inplace=True, errors='raise')
labeled_Rpeaks_nsr['Label'] = 0
labeled_Rpeaks_nsr

,Rpeaks,Label
0,2,0
1,104,0
2,258,0
3,414,0
4,568,0
...,...,...
111258,19446442,0
111259,19446610,0
111260,19446776,0
111261,19446938,0


In [5]:
Rpeaks_nsr = labeled_Rpeaks_nsr["Rpeaks"]
Label_nsr = labeled_Rpeaks_nsr["Label"]
Label_nsr = np.array(Label_nsr, dtype=np.float64)

#labeled_Rpeaks_N_AFIB[0:1000]
labeled_Rpeaks_nsr.isnull().values.any() ### Check for NaN values

False

In [6]:
def segmenting_record_nsr(seg_value_nsr):
    rri2_nsr = np.diff(Rpeaks_nsr) ### Storing the intervals between rpeaks
    #print(rri2.dtype)
    rri2_nsr = np.array(rri2_nsr, dtype=np.float64) ### So no overflow issues happens when calc in for loop later
    #print(rri2.dtype)
    #print(f"length before: {len(rri2)}")
    amount_nsr = -(len(rri2_nsr) % seg_value_nsr) #
    print(f"Amount to remove {amount_nsr}")
    rec_amount_nsr = rri2_nsr[:amount_nsr]
    #print(f"length before: {len(rec_amount)}")
    seg_shape_nsr = len(rec_amount_nsr) // seg_value_nsr #
    print(f"Shape 0: {seg_shape_nsr}")
    segmented_rec_nsr = rec_amount_nsr.reshape(seg_shape_nsr,seg_value_nsr) #
    return segmented_rec_nsr

In [7]:
segmented_record_nsr = segmenting_record_nsr(60)

Amount to remove -11
Shape 0: 30113


### X

In [8]:
features2_nsr = []

    ### Calculating features for every single segmented "block" inside the segmented_record variable
for q in range(len(segmented_record_nsr)):
    #MEAN
    ff1_nsr = np.nanmean(segmented_record_nsr[q]) ### nanmean, nanstd computes values while ignoring nan-values
    #STD
    ff2_nsr = np.nanstd(segmented_record_nsr[q])
    #RMSSD
    sum_nsr = 0
    for z in range(len(segmented_record_nsr[q]) - 1): ### loops 19 times
        sum_nsr += (segmented_record_nsr[q][z] - segmented_record_nsr[q][z+1])**2
    sum_multiplied_nsr = 1/(len(segmented_record_nsr) - 1) * sum_nsr
    ff3_nsr = np.sqrt(sum_multiplied_nsr)
    #NORMALIZED RMSSD
    ff4_nsr = (ff3_nsr / ff1_nsr)
    ### COVARIANCE
    #z = np.stack((ff1, ff2))
    #ff5 = np.cov(z) ### try cov with previous segment

    my_features2_nsr = pd.Series([np.around(ff1_nsr,3), np.around(ff2_nsr,3), np.around(ff3_nsr,3), np.around(ff4_nsr,3)], 
                            index=['Mean','STD','RMSSD','Normalized RMSSD'])
    features2_nsr.append(my_features2_nsr)

In [9]:
n_nsr = 0

print(f"Features for that specific segment:\n{features2_nsr[n_nsr]}") 
print('\n'f"features2 variable contains {len(features2_nsr)} segments")

Features for that specific segment:
Mean                160.400
STD                   8.830
RMSSD                 0.338
Normalized RMSSD      0.002
dtype: float64

features2 variable contains 30113 segments


### Y

In [10]:
y_amount_nsr = Label_nsr[:-12] # <=============== (1)
y_shape_nsr = y_amount_nsr.reshape(30113, 60) # <=============== (2)
y_list_nsr = []

### Calc if every 20 segment block is Normal synus rythm or AFIB
for g in range(len(y_shape_nsr)):
    #y_segment = y_shape
    sum_segment_nsr = np.sum(y_shape_nsr[g])
    if sum_segment_nsr >= 30: # <=============== (3)
        sum_segment_nsr = 1
    else: sum_segment_nsr = 0
    y_list_nsr.append(sum_segment_nsr)

y_list_nsr = np.array(y_list_nsr, dtype=np.float64)
print(y_list_nsr.dtype)
print(f"Length of y_list: {len(y_list_nsr)}")

float64
Length of y_list: 30113


# CYGWIN

In [19]:
# import os
# import subprocess #, shlex

# def cygwin(command):
#     """
#     Run a Bash command with Cygwin and return output.
#     """
#     # Find Cygwin binary directory
#     for cygwin_bin in [r'C:\cygwin\bin', r'C:\cygwin64\bin']:
#         if os.path.isdir(cygwin_bin):
#             break
#     else:
#         raise RuntimeError('Cygwin not found!')
#     # Make sure Cygwin binary directory in path
#     if cygwin_bin not in os.environ['PATH']:
#         os.environ['PATH'] += ';' + cygwin_bin
#     # Launch Bash
#     p = subprocess.Popen(
#         args=['bash', '-c', command],
#         stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#     p.wait()
#     # Raise exception if return code indicates error
#     if p.returncode != 0:
#         raise RuntimeError(p.stderr.read().rstrip())
#     # Remove trailing newline from output
#     return (p.stdout.read() + p.stderr.read()).rstrip()

In [20]:
# cygwin('pwd')

In [21]:
# cygwin('rdann -r 04936 -f 0 -a atr -v > data/04936_atr.txt')

# command_atr_new = "rdann -r 04936 -f 0 -a atr -v > /data/04936_atr.txt"
# command_atr = "rdann -r 04936 -f 0 -a atr -v > C:/Users/masud/Desktop/Github_Bachelor_Project/wfdb-python/data/04936_atr.txt"